In [ ]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

API_KEY = "bob_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

In [ ]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 3. connect to SP

In [ ]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

In [ ]:
#Paste in invitation from Carol agent
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '40530d18-810b-4c46-a984-8c12792dcc8b', 'serviceEndpoint': 'http://192.168.65.3:8020', 'label': 'Alice', 'recipientKeys': ['7pfyv4kVtMNHvZRzN18nqd7rRzDFSXeYa2B8Z8ne3Zkz']}

In [ ]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Carol's connection ID
print("Connection", response)
sp_connection = response["connection_id"]


In [ ]:
# sp_connection = '73437da8-d87e-46b7-aa1e-c1ccc86fccf7'

### switch to SP

In [ ]:
# Print connection list
connection = await agent_controller.connections.get_connection(sp_connection)
print("SP AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

In [ ]:
response = await agent_controller.connections.get_connections()
print(response)

## 5. connect to DM

In [ ]:
#Paste in invitation from Carol agent
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'da9e0fdd-282d-47b6-bced-dcc587229941', 'serviceEndpoint': 'http://192.168.65.3:8020', 'label': 'Alice', 'recipientKeys': ['8VYroiwWGJ4DtfngCqf1R3Dz1QsVZ4dyCo2mUCRfo7rR']}

In [ ]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Carol's connection ID
print("Connection", response)
dm_connection = response["connection_id"]
print(dm_connection)

### switch to DM

In [ ]:
connection = await agent_controller.connections.get_connection(dm_connection)
print(connection)
print("Is Active?", connection["state"])

## 8. exchange DID with DM

In [ ]:
my_did = await agent_controller.wallet.get_public_did()
my_did = my_did['result']['did']
print("my DID", my_did)
dm_did = 'None'

In [ ]:
def messages_handler(payload):
    global dm_did
    connection_id = payload["connection_id"]
    print("Handle message", payload, connection_id)
    dm_did = payload['content']
    print('DM DID', dm_did)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

In [ ]:
await agent_controller.messaging.trust_ping(dm_connection, "hello, world!")

In [ ]:
response = await agent_controller.messaging.send_message(dm_connection, 'music_'+my_did)
print(response)

In [ ]:
print(dm_did)

## present proof that itself belongs to DM to SP

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")
          
proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}
agent_controller.register_listeners([proof_listener], defaults=True)

### switch to SP

In [ ]:
response = await agent_controller.proofs.get_records()
print((response['results']))

print('\n')

state = response['results'][0]["state"]
presentation_exchange_id = response['results'][0]['presentation_exchange_id']
presentation_request = response['results'][0]['presentation_request']

print('Presentation Exchange ID\n')
print(response['results'][0]['presentation_exchange_id'])
print('Presentation Request Object\n')
print(response['results'][0]['presentation_request'])
print('Requested Attributes\n')
print(response['results'][0]['presentation_request']['requested_attributes'])
requested_attribs = response['results'][0]['presentation_request']['requested_attributes']
print('Requested Predicates\n')
print(response['results'][0]['presentation_request']['requested_predicates'])
requested_predicates = response['results'][0]['presentation_request']['requested_predicates']

In [ ]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print(credentials)

if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }
    else:
        self_attested[referent] = "South Africa"

for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
proof = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(proof)
print("\nXXX")
print(predicates)
print(revealed)
print(self_attested)

In [ ]:
response = await agent_controller.proofs.send_presentation(presentation_exchange_id, proof)
print(response)

## save new VC

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

In [ ]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

In [ ]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

### switch to DM

In [ ]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "Credential of Ownership")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

## ask proof of ownership from SP

In [ ]:
print("Request proof from SP")
#Set some variables

revocation = False
SELF_ATTESTED = True
exchange_tracing = False

#Enable this to ask for attributes to identity a user
req_attrs = [
    {"name": "possession", "restrictions": [{"issuer_did": dm_did}]},
    {"name": "time", "restrictions": [{"issuer_did": dm_did}]},
]

if revocation:
    req_attrs.append(
        {
            "name": "owner",
            "restrictions": [{"issuer_did": agent.did}],
            "non_revoked": {"to": int(time.time() - 1)},
        },
    )



#Set predicates for Zero Knowledge Proofs
req_preds = []

indy_proof_request = {
    "name": "Proof of Ownership",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}

if revocation:
    indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": sp_connection,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

In [ ]:
response = await agent_controller.proofs.send_request(proof_request_web_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)

### switch to SP

In [ ]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
print(verify)

In [ ]:
print(verify['state'])
print(verify['state'] == 'verified')

In [ ]:
for (name, val) in verify['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print(name + " : " + val['raw'])

In [ ]:
response = await agent_controller.terminate()
print(response)